In [2]:
import arcpy
import numpy as np


ModuleNotFoundError: No module named 'arcpy'

In [ ]:
huc = '04120104'
watershed_boundary = r"C:\Users\smcavoy\Documents\ArcGIS\Packages\archydro_04120104_A069F740-ADE6-4E5A-A7CB-7538AA6503C9\commondata\105_huc04120104\HUC_Boundary.shp"
dem_mask = r"C:\Users\smcavoy\Documents\ArcGIS\Packages\archydro_04120104_A069F740-ADE6-4E5A-A7CB-7538AA6503C9\commondata\raster_data1\dem_3dep_mask.tif"
fac = r"C:\Users\smcavoy\Documents\ArcGIS\Packages\archydro_04120104_A069F740-ADE6-4E5A-A7CB-7538AA6503C9\commondata\raster_data\w1_fac"
catchment = r"C:\Users\smcavoy\Documents\ArcGIS\Packages\archydro_04120104_A069F740-ADE6-4E5A-A7CB-7538AA6503C9\v107\archydro_04120104.gdb\W1_Catchment"
Str = r"C:\Users\smcavoy\Documents\ArcGIS\Packages\archydro_04120104_A069F740-ADE6-4E5A-A7CB-7538AA6503C9\commondata\raster_data\w1_str"
dem_fill = r"C:\Users\smcavoy\Documents\ArcGIS\Packages\archydro_04120104_A069F740-ADE6-4E5A-A7CB-7538AA6503C9\commondata\raster_data\w1_fil"
fdrstradj = r"C:\Users\smcavoy\Documents\ArcGIS\Packages\archydro_04120104_A069F740-ADE6-4E5A-A7CB-7538AA6503C9\commondata\raster_data\w1_fdrstradj"
county_boundary = r"Y:\CCSI\TECH\NYS DEC\SB299\SC813 WO11_CRRA_Support\ANALYSIS\Task_Hybrid\102_Predictors\104_Boundaries\101_Counties\Boundaries_TwoCounties_UTM18.shp"

fp_0_2pct = r"P:\02\NY\R2_BLE_Discovery\DELIVERABLES\04120104_Niagara\FloodplainMapping\Spatial_Files\BLE_04120104.gdb\FP_0_2PCT"
fp_01pct = r"P:\02\NY\R2_BLE_Discovery\DELIVERABLES\04120104_Niagara\FloodplainMapping\Spatial_Files\BLE_04120104.gdb\BLE_WSE01PCT"
water_inventory = r"Y:\CCSI\TECH\NYS DEC\SB299\SC813 WO11_CRRA_Support\ANALYSIS\Task_Hybrid\102_Predictors\101_Predictor_Vars\WaterInventory_PriorityWaterbodies\nysdec_wipwl\c04120104_Lakes_Streams.tif"

waterbodies = r"C:\Users\smcavoy\Documents\CRRA_Working_FlowDir\NHD_H_New_York_State_GDB\NHD_H_New_York_State_GDB.gdb\NHDWaterbody"
prism = r"Y:\CCSI\TECH\NYS DEC\SB299\SC813 WO11_CRRA_Support\ANALYSIS\Task_Hybrid\102_Predictors\101_Predictor_Vars\PRISM\PRISM_ppt_30yr_normal_4kmM2_annual_asc\PRISM_ppt_30yr_normal_4kmM2_annual_asc.asc"
natsgo = r"Y:\CCSI\TECH\NYS DEC\SB299\SC813 WO11_CRRA_Support\ANALYSIS\Task_Hybrid\102_Predictors\101_Predictor_Vars\NATSGO\MapunitRaster_10m_UTM18N.tif"
nlcd = r"Y:\CCSI\TECH\NYS DEC\SB299\SC813 WO11_CRRA_Support\ANALYSIS\Task_Hybrid\102_Predictors\101_Predictor_Vars\NLCD2016\nlcd2016_ny_UTM18NAD_clipped_10m.tif"


workspace = r"C:\Users\smcavoy\Documents\CRRA_Working_FlowDir\ArcHydroDone\HUC104\postArcHydro"

arcpy.env.workspace = workspace
arcpy.env.snapRaster = dem_mask
arcpy.env.cellSize = dem_mask
arcpy.env.outputCoordinateSystem = dem_mask
arcpy.env.extent = dem_mask 
arcpy.env.overwriteOutput = True 

In [ ]:
#step 12
#Raster calculator to converrt the flow accumulation values into square miles
drainageArea = arcpy.sa.Raster(fac) / 25900.0
drainageArea.save(f"{huc}_DrainageArea_sqmi.tif")

In [ ]:
#step 13
#Zonal statistics on the drainage area sq mi with zones = Catchment polygons
zonalst = arcpy.sa.ZonalStatistics(catchment, "GridID", drainageArea, "MAXIMUM")
zonalst.save(f"{huc}_Catchment_DAsqmi.tif")

In [ ]:
#step 16
#raster calculator 
streams_calc = arcpy.sa.SetNull(drainageArea, Str, "Value < 1")
streams_calc.save("streams_setnull.tif")

In [ ]:
#step 16.5
#raster to polyline the trimmed stream grid
stream_trim = arcpy.RasterToPolyline_conversion(streams_calc, "Str_onesqmi.shp")
arcpy.CopyFeatures_management("Str_onesqmi.shp", workspace+f"\{huc}_Str_onesqmi.shp")

In [ ]:
#step 17
#only select NHD waterbodies f they intersect streams; only include Estuary, Reservior, LakePond
wb_clip = arcpy.Clip_analysis(waterbodies, watershed_boundary, "wb_clip.shp" )

wb_select1 = arcpy.SelectLayerByAttribute_management(wb_clip, "NEW_SELECTION", 
                                                     "FTYPE = 390 OR FTYPE = 436 OR FTYPE = 493")

wb_copy = arcpy.CopyFeatures_management(wb_select1, "wb_select1.shp")

wb_select2 = arcpy.SelectLayerByLocation_management(wb_copy, 
                                                    "INTERSECT", 
                                                    workspace+f"\{huc}_Str_onesqmi.shp")
wb = arcpy.CopyFeatures_management(wb_select2, f"{huc}_waterbodies.shp")

In [ ]:
#step 18
#turn waterbodies polygon into a raster
wb_addfield = arcpy.AddField_management(wb, "IS_water", "SHORT")
wb_calcfield = arcpy.CalculateField_management(wb_addfield, "IS_water", '1')

wb_raster = arcpy.PolygonToRaster_conversion(wb_calcfield, "IS_water", workspace+f"\c{huc}_wb" )

In [ ]:
#mosaic the wb raster with the streams
streams_lakes = arcpy.MosaicToNewRaster_management([arcpy.sa.Raster(streams_calc), arcpy.sa.Raster(wb_raster)], 
                                                   workspace, f"{huc}_Str_ip.tif",
                                                  number_of_bands=1)
sl_full = arcpy.sa.Con(arcpy.sa.IsNull(streams_lakes), 0, streams_lakes)
sl_mask = arcpy.sa.ExtractByMask(sl_full, watershed_boundary)
sl_mask.save(f"{huc}_Str_Final.tif")

In [ ]:
#step 20a
#identify subcatchments where BLE has been done
catchment_copy = arcpy.CopyFeatures_management(catchment, "catchment_copy.shp")
#catchment_addfield = arcpy.AddField_management(catchment_copy, "hasBLE", "SHORT")
catchment_calcfield1 = arcpy.CalculateField_management(catchment_addfield, "hasBLE", "0")

catchment_select = arcpy.SelectLayerByLocation_management(catchment_calcfield1, "INTERSECT", fp_0_2pct)
catchment_calcfield2 = arcpy.CalculateField_management(catchment_select, "hasBLE", "1")

#create a raster on field hasBLE
hasBLE = arcpy.PolygonToRaster_conversion("catchment_copy.shp", "hasBLE", "hasBLE")
arcpy.sa.Raster("hasBLE").save(f"c{huc}_hasBLE.tif")

In [ ]:
#step 20b
#calculate horizontal and vertical overland flow distance
#flow distance tool
void = arcpy.sa.FlowDistance(streams_lakes, dem_fill, distance_type= "VERTICAL")
void.save(f"{huc}_vofd.tif")

hoid = arcpy.sa.FlowDistance(streams_lakes, dem_fill, distance_type= "HORIZONTAL")
hoid.save(f"{huc}_hofd.tif")

In [ ]:
#step 21
#stream order; use NOT truncated, NOT with lakes stream raster
stream_order = arcpy.sa.StreamOrder(arcpy.sa.Raster(Str), fdrstradj, "STRAHLER")
stream_order.save(f"{huc}_stream_order_str.tif")

In [ ]:
#step 22
#map stream order to subcatchment grid with zonal stats zones = Catchment
stream_order_grid = arcpy.sa.ZonalStatistics(catchment, "GridID", stream_order, "MAXIMUM")
stream_order_grid.save(f"{huc}_stream_order_grid.tif")

In [ ]:
#step 22
#PRISM Rainfall data ; resample first and then extract by mask

prism_resample = arcpy.Resample_management(prism, "prism_bbox", 
                                           "10", "BILINEAR")

prism_mask = arcpy.sa.ExtractByMask("prism_bbox", watershed_boundary)
prism_mask.save(f"{huc}_prism_final.tif")

In [ ]:
#step 23
#NATSGO - national soil data; map unit raster_10m
natsgo_mask = arcpy.sa.ExtractByMask(arcpy.sa.Raster(natsgo), watershed_boundary)
natsgo_mask.save(f"{huc}_natsgo_final.tif")

In [ ]:
#step 24
#national land cover ; nlcd2016_utm1bnd_clipped10m
nlcd_mask = arcpy.sa.ExtractByMask(nlcd, watershed_boundary)
nlcd_mask.save(f"{huc}_nlcd_final.tif")


In [ ]:
#step 25
#water inventory; reclassify and then extract by mask
remap = arcpy.sa.RemapRange([[0, 0, 0], [0, 999999999999, 1]])
wi_reclass = arcpy.sa.Reclassify(arcpy.sa.Raster(water_inventory), "VALUE", remap)

wi_mask = arcpy.sa.ExtractByMask(wi_reclass, watershed_boundary)
wi_mask.save(f"{huc}_NYSDEC_StrLakes_Final.tif")

In [ ]:
#step 26
#100 yr fp raster; resample, reclassify into binary, extract by mask
resample100 = arcpy.Resample_management(arcpy.sa.Raster(fp_01pct), "fp_01pct_res", "10",
                                        "BILINEAR")

remap2 = arcpy.sa.RemapRange([["NoData", "NoData", 0], [0, 9999999999, 1]])
reclass100 = arcpy.sa.Reclassify(resample100, "VALUE", remap2)
reclass100.save("fp_01pct_reclass.tif")




In [ ]:
#might have to refresh top cell block with workspace folder
mask100 = arcpy.sa.ExtractByMask("fp_01pct_reclass.tif", watershed_boundary)
mask100.save(f"{huc}_fp_01pct_final.tif")

In [ ]:
#step 27
#run slope tool on dem mask
slope = arcpy.sa.Slope(arcpy.sa.Raster(dem_mask), "DEGREE")
slope.save(f"{huc}_slope_Final.tif")

In [ ]:
#step 28
#focal statistics window on dem mask
focal = arcpy.sa.FocalStatistics(arcpy.sa.Raster(dem_mask), arcpy.sa.NbrRectangle(5, 5, "CELL"), "STD")
focal.save(f"{huc}_dem_5x5STD.tif")


In [ ]:
#step 29 - refresh top cell block
#wetness topographic index; series of raster calculations
slope_raster = arcpy.sa.Raster(workspace + f"\\{huc}_slope_Final.tif")
slope_rad = (slope_raster * np.pi / 2.0) / 90.0

con1 = arcpy.sa.Con(slope_rad > 0, arcpy.sa.Tan(slope_rad), .001)
tan_slope_deg = arcpy.sa.Con(con1 > 0, con1, .001)

fac_scaled = (arcpy.sa.Raster(fac) + 1) * 10.0 * 10.0

final_calc = arcpy.sa.Ln(fac_scaled / tan_slope_deg)
final_calc.save(f"{huc}_WTI_Final.tif")
